## Out.txt

In [1]:

import json
import pandas as pd
import numpy as np


# CHANNEL_LIST_FILE = "data/channel_list_merged/finance_kw_10k_sub_+.csv"
# CHANNEL_QUALITY_FILE = "data/channel_quality/finance_kw_10k_sub_+_filter_0-200.jsonl"

CHANNEL_LIST_FILE = "data/channel_list_merged/starngage_5000.csv"
# CHANNEL_QUALITY_FILE = "data/channel_quality/starngage_5000_filter_0-100.jsonl"
# CHANNEL_QUALITY_FILE = "data/channel_quality/starngage_5000_filter_300-600.jsonl"
CHANNEL_QUALITY_FILE = "data/channel_quality/starngage_5000_filter_1000-1400.jsonl"

In [2]:
df = pd.read_csv(CHANNEL_LIST_FILE)

# # remove duplicates
df = df.drop_duplicates()
len(df[df['n_subs'] > 10000])

4528

In [3]:
rs = []
with open(CHANNEL_QUALITY_FILE, "r") as f:
    r = f.readlines()
    for i in r:
        # rs.append(json.loads(i.replace("'", "\"").replace("False", "false").replace("True", "true")))
        rs.append(json.loads(i))

In [4]:
#     n_total = len(r['snrss'])
#     n_snr_ok = (np.array(r['snrss']) > 20).sum()
#     # n_ac_ok = (np.array(r['acss'])).sum()

from audio_ac import ac_get_speech_probs

rs_df = pd.DataFrame(rs)
rs_df["n_total"] = rs_df.apply(lambda x: len(x['snrss']), axis=1)
rs_df['n_srn_ok'] = rs_df.apply(lambda x: (np.array(x['snrss']) > 20).sum(), axis=1)
rs_df['n_ac_ok'] = rs_df.apply(lambda x: len([i for i in ac_get_speech_probs(x['acss']) if i > 0.9]), axis=1)

max_n_snr_ok = rs_df['n_srn_ok'].max()
max_n_ac_ok = rs_df['n_ac_ok'].max()
rs_df["score"] = rs_df.apply(lambda x: x['n_srn_ok'] / max_n_snr_ok + x['n_ac_ok'] / max_n_ac_ok, axis=1)

rs_df.drop(columns=['snrss', 'acss'], inplace=True)

d:\Projects\SE-datapipeline\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# rs_df inner join df
df = df.merge(rs_df, left_on='url', right_on='url', how='inner')

# move ulr column to the last
cols = df.columns.tolist()
cols.remove('url')
cols.append('url')
df = df[cols]

In [6]:
df.sort_values(by='score', ascending=False, inplace=True)

In [7]:
df.to_csv("tmp.csv", index=False)

In [8]:
# def clean_channel_cond(r):
#     n_total = len(r['snrss'])
#     n_snr_ok = (np.array(r['snrss']) > 20).sum()
#     # n_ac_ok = (np.array(r['acss'])).sum()
#     ac_speech_probs = []
#     for ac_seg in r['acss']:
#         for item in ac_seg:
#             if item['label'] == 'Speech':
#                 ac_speech_probs.append(item['score'])
#     n_ac_ok = (np.array(ac_speech_probs) > 0.9).sum()

#     mean_snr = np.mean(r['snrss'])
#     mean_ac = np.mean(ac_speech_probs)

#     is_good = n_total > 0 and n_snr_ok / n_total > 0.1 and n_ac_ok / n_total > 0.1
#     return is_good, {
#         'n_total': n_total,
#         'n_snr_ok': n_snr_ok,
#         'n_ac_ok': n_ac_ok,
#         'mean_snr': mean_snr,
#         'mean_ac': mean_ac,
#         'url': r['url'],
#     }

# r_ok = []
# n_ok = []
# r_not_ok = []
# n_not_ok = []
# for r in rs:
#     is_good, info = clean_channel_cond(r)
#     if is_good:
#         r_ok.append(r)
#         n_ok.append(info)
#     else:
#         r_not_ok.append(r)
#         n_not_ok.append(info)

In [9]:
# ok_df = df[df['url'].isin([r['url'] for r in r_ok])]
# not_ok_df = df[df['url'].isin([r['url'] for r in r_not_ok])]

# # drop duplicate title
# ok_df = ok_df.drop_duplicates(subset=['title'])
# not_ok_df = not_ok_df.drop_duplicates(subset=['title'])

# pass

In [10]:
# len(ok_df)
# not_ok_df
# rs[1]
# n_not_ok

In [11]:
# len(not_ok_df)

In [12]:
# # join n_total, n_snr_ok, n_ac_ok
# # ok_df = pd.merge(ok_df, pd.DataFrame(n_ok), left_on='url', right_on='url')
# not_ok_df = pd.merge(not_ok_df, pd.DataFrame(n_not_ok), left_on='url', right_on='url')

# # move column 'url' to the last
# cols = list(ok_df.columns)
# cols.remove('url')
# cols.append('url')
# ok_df = ok_df[cols]
# not_ok_df = not_ok_df[cols]

In [13]:
# ok_df.sort_values(by='mean_snr', ascending=False)

In [14]:
# # not_ok_df order by mean snr 
# # not_ok_df.sort_values(by='mean_ac', ascending=False).sort_values(by='mean_snr', ascending=False).head(20)
# not_ok_df.sort_values(by='n_ac_ok', ascending=False).head(50)


In [15]:
# all_df = pd.concat([ok_df, not_ok_df])
# all_df['score'] = all_df['n_ac_ok'] / all_df['n_total']

In [16]:
# ok_df.to_csv('tmp_16_channel_ok.csv', index=False)

In [17]:
# # move column 'url' to the last
# cols = list(ok_df.columns)
# cols.remove('url')
# cols.append('url')
# ok_df = ok_df[cols]
# not_ok_df = not_ok_df[cols]

# ok_df.to_csv('channel_clean.csv', index=False)
# not_ok_df.to_csv('channel_noise.csv', index=False)